<a href="https://colab.research.google.com/github/A-Burnhard/OpenAI-Langchain-function-calling/blob/main/OpenAI_%26_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai
!pip install json
!pip install -U langchain-openai

In [12]:
import langchain
import openai
import json

# Environment Variables
import os





In [13]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from langchain.tools import format_tool_to_openai_function, YouTubeSearchTool, MoveFileTool

In [14]:
function_description = {
    "name": "send_sms",
    "description": "Send an SMS to a specified phone number with a given message.",
    "parameters": {
        "type": "object",
        "properties": {
            "number": {
                "type": "string",
                "description": "The phone number of the recipient (e.g., '0550916600').",
            },
            "message": {
                "type": "string",
                "description": "The content of the SMS message.",
            },
        },
        "required": ["number", "message"],
    },
}

In [1]:
import requests  # Assuming you have the requests library installed

def send_sms(number, message):
    """
    Sends an SMS to the specified phone number with the given message.

    Parameters:
    - number (str): The phone number of the recipient (e.g., "0550916600").
    - message (str): The content of the SMS.

    Returns:
    - dict: A dictionary containing the result of the SMS sending operation.
    """
    # Construct the payload for the POST request
    payload = {
        "number": number,
        "message": message,
    }

    # Make a POST request to the specified URL
    url = "https://laws.adudor.com/api/fire-sms"
    response = requests.post(url, json=payload)

    # Return the result as a dictionary
    result = {
        "status_code": response.status_code,
        "response_content": response.json() if response.headers['content-type'] == 'application/json' else response.text,
    }

    return result

In [15]:
openai.api_key = ''
llm = ChatOpenAI(model="gpt-4-0613", openai_api_key=openai.api_key)

In [17]:
user_request = """ send an sms to 0550916600 """

In [18]:
first_response = llm.predict_messages([HumanMessage(content=user_request)],
                                      functions=function_description)
first_response

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-4vqnJ***************************************XiT9. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
first_response.additional_kwargs

{}